In [88]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import math

In [89]:
tickers = ["NVDA", "NFLX", "GME", "BB", "NOK", "AMZN", "TSLA"]
etfs = ["SPY", "IWM", "DIA"]

class DataDownloader:
    def __init__(self, tickers, etfs):
        self.tickers = tickers
        self.etfs = etfs

    def download(self, period='1y'):
        return yf.download(self.tickers + self.etfs, period=period)

    def quarterly(self):
        return self.download(period='3mo')

    def annual(self):
        return self.download(period='1y')
    
    def decade(self):
        return self.download(period='10y')

# Usage:
downloader = DataDownloader(tickers, etfs)
pre_quarterly_data = downloader.quarterly()
pre_annual_data = downloader.annual()
pre_decade_data = downloader.decade()

[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


In [90]:
# I have this cell so that when I want to call back to the default df i can use prefix(pre_) for future use.
quarterly_data= pre_quarterly_data['Adj Close']
annual_data= pre_annual_data['Adj Close']
decade_data= pre_decade_data['Adj Close']

In [91]:
annual_data.tail()

,AMZN,BB,DIA,GME,IWM,NFLX,NOK,NVDA,SPY,TSLA
Date,,,,,,,,,,
2023-09-13,144.850006,5.51,345.635925,17.809999,182.970001,412.239990,4.010,454.850006,444.940552,271.299988
2023-09-14,144.720001,5.38,349.016998,18.330000,185.550003,400.489990,4.020,455.809998,448.777008,276.040009
2023-09-15,140.389999,5.39,346.140015,18.219999,183.610001,396.940002,3.930,439.000000,443.369995,274.390015
2023-09-18,139.979996,5.22,346.149994,17.549999,182.389999,394.399994,3.950,439.660004,443.630005,265.279999
2023-09-19,137.218399,5.16,344.410004,17.455000,181.669998,396.370087,4.005,435.885986,442.070007,266.144989


# Part 1: We'll create an index with our Tickers:
           


In [92]:
table = pd.DataFrame(index=tickers)
table

""
NVDA
NFLX
GME
BB
NOK
AMZN
TSLA


# Part 2: We'll calculate weight using the following formula:
           
###   Price / Total Price

In [93]:
# Taking the last row of quartely_data.tail(), for most current price. Then summed to calculate portfolio value...
# assuming that each stock is bought only 1 time.
current_prices = quarterly_data[tickers].iloc[-1]
summed_prices = current_prices.sum()


#Each value in the row 'current_price' will be calculated to append it's weight % to a new column 
weight_list = []
for price in current_prices:
    weight = (price/summed_prices) * 100
    weight_list.append(str(round(weight,2)) + "%")

table['Weight'] = weight_list
table

,Weight
NVDA,34.53%
NFLX,31.4%
GME,1.38%
BB,0.41%
NOK,0.32%
AMZN,10.87%
TSLA,21.09%


# Part 3: We'll calculate 3-month trailing Annualized Volatility using the following formula:
           
### ( STD [ Daily Return ] )( SQRT [ Trading Days] ) * 100

In [94]:
def calculate_annualized_volatility(ticker_data):
    daily_return = ticker_data.pct_change().dropna()
    std_dev = daily_return.std()
    quarter_trading_days = daily_return.count()
    annualized_volatility = std_dev * math.sqrt(quarter_trading_days) * 100
    return annualized_volatility 

# avs- annualized volatilities 
avs = {}
for ticker in tickers:
    avs[ticker] = calculate_annualized_volatility(quarterly_data[ticker])
    
#Create Annualized votality column
for stock, av in avs.items():    
    table.loc[stock, 'Annualized Volatility'] = (str(round(av,2)) + "%")

In [95]:
table

,Weight,Annualized Volatility
NVDA,34.53%,19.97%
NFLX,31.4%,17.67%
GME,1.38%,21.12%
BB,0.41%,35.05%
NOK,0.32%,12.69%
AMZN,10.87%,15.39%
TSLA,21.09%,26.59%


In [96]:
annual_data

,AMZN,BB,DIA,GME,IWM,NFLX,NOK,NVDA,SPY,TSLA
Date,,,,,,,,,,
2022-09-19,124.660004,5.74,304.048523,28.959999,177.617126,243.630005,4.488474,133.738205,382.528137,309.070007
2022-09-20,122.190002,5.58,300.980072,27.540001,175.196701,242.850006,4.402158,131.679443,378.137268,308.730011
2022-09-21,118.540001,5.45,295.872589,26.860001,172.520416,236.869995,4.315841,132.528931,371.541107,300.799988
2022-09-22,117.309998,5.24,294.608002,24.700001,168.633957,237.050003,4.258296,125.533211,368.420288,288.589996
2022-09-23,113.779999,5.07,290.039673,25.040001,164.619553,226.410004,4.085663,125.083488,362.247437,275.329987
...,...,...,...,...,...,...,...,...,...,...
2023-09-13,144.850006,5.51,345.635925,17.809999,182.970001,412.239990,4.010000,454.850006,444.940552,271.299988
2023-09-14,144.720001,5.38,349.016998,18.330000,185.550003,400.489990,4.020000,455.809998,448.777008,276.040009
2023-09-15,140.389999,5.39,346.140015,18.219999,183.610001,396.940002,3.930000,439.000000,443.369995,274.390015


# Part 4-6: 12-month trailing beta agaisnt's etfs:
           


# https://www.investopedia.com/terms/b/beta.asp

## Beta is equal to the :
### Covariance divided by variance.
#### Covariance = (returns on individual stocks agaisnt returns of ETF's).cov
#### Variance = (returns on ETF's).var

In [97]:
# First define daily returns
returns = annual_data.pct_change().dropna()

# Function to calculate beta
def calculate_beta(stock, etf):
    covariance = returns[stock].cov(returns[etf])
    variance = returns[etf].var()
    beta = covariance / variance
    return beta

# Calculate beta for each stock against each ETF and append to table
for etf in etfs:
    beta_column = []
    for stock in tickers:
        beta = calculate_beta(stock, etf)
        beta_column.append(round(beta,2))
    table[f'Beta against {etf}'] = beta_column


In [98]:
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA
NVDA,34.53%,19.97%,2.08,1.29,1.81
NFLX,31.4%,17.67%,1.45,0.99,1.43
GME,1.38%,21.12%,1.61,1.44,1.62
BB,0.41%,35.05%,1.59,1.20,1.52
NOK,0.32%,12.69%,1.12,0.84,1.23
AMZN,10.87%,15.39%,1.50,0.98,1.32
TSLA,21.09%,26.59%,1.77,1.31,1.44


# Part 7 : Average Weekly Drawdown

In [99]:

# Using pre_annual because annual_data wont work as it uses [Adj Close], not [High] & [Low]
highs = pre_annual_data['High'].pct_change().dropna()
lows = pre_annual_data['Low'].pct_change().dropna()
awd = []
for ticker in tickers:
    avg_highs = highs[ticker].mean()
    avg_lows = lows[ticker].mean()
    avg_weekly_drawdown = ((avg_lows - avg_highs) / avg_highs) * 100 
    awd.append(round(avg_weekly_drawdown, 2))

table['Average Weekly Drawdown'] = awd


# Part 8 : Maximum Weekly Drawdown

In [100]:
mwd = []
for ticker in tickers:
    max_highs = highs[ticker].max()
    min_lows = lows[ticker].min()
    max_weekly_drawdown = ((min_lows - max_highs) / max_highs) * 100
    mwd.append(round(max_weekly_drawdown, 2))
    
table['Maximum Weekly Drawdown'] = mwd



In [101]:
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown
NVDA,34.53%,19.97%,2.08,1.29,1.81,-0.13,-128.46
NFLX,31.4%,17.67%,1.45,0.99,1.43,2.89,-176.71
GME,1.38%,21.12%,1.61,1.44,1.62,29.50,-142.06
BB,0.41%,35.05%,1.59,1.20,1.52,-7.75,-207.67
NOK,0.32%,12.69%,1.12,0.84,1.23,-21.66,-319.96
AMZN,10.87%,15.39%,1.50,0.98,1.32,1.07,-203.87
TSLA,21.09%,26.59%,1.77,1.31,1.44,-602.20,-205.75


# Part 9: Total Return

In [102]:
total_return_list = []
for ticker in tickers:
    current_value = decade_data[ticker].iloc[-1]
    initial_value = decade_data[ticker].dropna().iloc[0]
    difference = current_value - initial_value
    total_return = (difference / initial_value) * 100
    total_return_list.append(str(round(total_return,2)) + "%")

table['Total Return'] = total_return_list
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return
NVDA,34.53%,19.97%,2.08,1.29,1.81,-0.13,-128.46,11563.83%
NFLX,31.4%,17.67%,1.45,0.99,1.43,2.89,-176.71,808.21%
GME,1.38%,21.12%,1.61,1.44,1.62,29.50,-142.06,88.03%
BB,0.41%,35.05%,1.59,1.20,1.52,-7.75,-207.67,-50.95%
NOK,0.32%,12.69%,1.12,0.84,1.23,-21.66,-319.96,-20.95%
AMZN,10.87%,15.39%,1.50,0.98,1.32,1.07,-203.87,779.45%
TSLA,21.09%,26.59%,1.77,1.31,1.44,-602.20,-205.75,2144.09%


# Part 10: Annualized Total Return

In [103]:
atr = []
for ticker in tickers:
    current_value = decade_data[ticker].iloc[-1]
    initial_value = decade_data[ticker].dropna().iloc[0]
    annualized_total_return = (((current_value / initial_value)**1/10)-1) * 100
    atr.append(str(round(annualized_total_return,2)) + "%")
table['Annualized Total Return'] = atr
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return
NVDA,34.53%,19.97%,2.08,1.29,1.81,-0.13,-128.46,11563.83%,1066.38%
NFLX,31.4%,17.67%,1.45,0.99,1.43,2.89,-176.71,808.21%,-9.18%
GME,1.38%,21.12%,1.61,1.44,1.62,29.50,-142.06,88.03%,-81.2%
BB,0.41%,35.05%,1.59,1.20,1.52,-7.75,-207.67,-50.95%,-95.1%
NOK,0.32%,12.69%,1.12,0.84,1.23,-21.66,-319.96,-20.95%,-92.09%
AMZN,10.87%,15.39%,1.50,0.98,1.32,1.07,-203.87,779.45%,-12.06%
TSLA,21.09%,26.59%,1.77,1.31,1.44,-602.20,-205.75,2144.09%,124.41%


In [104]:
risks = pd.DataFrame(index= etfs)

# Part 1: Index

In [105]:
risks

""
SPY
IWM
DIA


# Part 2: Correlation agaisnt ETF's

In [106]:

annual_data_change = annual_data.pct_change()
annual_data_complete = annual_data_change.drop(annual_data_change.index[0])


corr_against_etf_list = []
for etf in etfs:
    annual_data_complete['portfolio'] = annual_data_complete[tickers].sum(axis=1) 
    corr_against_etf = annual_data_complete[etf].corr(annual_data_complete['portfolio'])
    corr_against_etf_list.append(round(corr_against_etf,2))

risks['Correlation against ETF'] = corr_against_etf_list
risks


,Correlation against ETF
SPY,0.79
IWM,0.72
DIA,0.62


# Part 3: Covariance agaisnt ETF's

In [107]:
cov_against_etf_list = []
for etf in etfs:
    annual_data_complete['portfolio'] = annual_data_complete[tickers].sum(axis=1) 
    cov_against_etf = annual_data_complete[[etf , 'portfolio']].cov()
    cov_against_etf_list.append(round(cov_against_etf[etf]['portfolio']*100,2))

risks['Covariance against ETF'] = cov_against_etf_list
risks

,Correlation against ETF,Covariance against ETF
SPY,0.79,0.13
IWM,0.72,0.16
DIA,0.62,0.09


In [108]:
decade_data[tickers].transpose().sum() / decade_data['BB'].count()

Date
2013-09-19    0.039616
2013-09-20    0.039390
2013-09-23    0.038640
2013-09-24    0.038929
2013-09-25    0.038806
                ...   
2023-09-13    0.520687
2023-09-14    0.518391
2023-09-15    0.507851
2023-09-18    0.502996
2023-09-19    0.501511
Length: 2517, dtype: float64

# Part 4: Tracking Error (10 years)


In [109]:
decade_data['SPY'].pct_change().sum()

1.2800545567128756

In [110]:
decade_data.pct_change().index[0]

Timestamp('2013-09-19 00:00:00')

In [111]:
decade_data_ch = decade_data[tickers].pct_change()
benchmark = decade_data_ch.drop(decade_data_ch.index[0])
benchmark.sum()

NVDA    5.816586
NFLX    3.221913
GME     5.578054
BB      0.960020
NOK     0.495989
AMZN    2.722324
TSLA    4.689806
dtype: float64

In [112]:
benchmark.sum()

NVDA    5.816586
NFLX    3.221913
GME     5.578054
BB      0.960020
NOK     0.495989
AMZN    2.722324
TSLA    4.689806
dtype: float64

In [113]:


TE_lists = []
for etf in etfs:
    ETF_return = decade_data[etf].pct_change().sum()
    TE =  (((benchmark.sum().mean() - ETF_return).std() * 100)* np.sqrt(decade_data['SPY'].count()))
    TE_lists.append(round(TE,2))
    
risks['Tracking Error'] =  TE_lists

In [114]:
risks

,Correlation against ETF,Covariance against ETF,Tracking Error
SPY,0.79,0.13,0.0
IWM,0.72,0.16,0.0
DIA,0.62,0.09,0.0


In [115]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Tickers
tickers = ["NVDA", "NFLX", "GME", "BB", "NOK", "AMZN", "TSLA"]
etfs = ["SPY", "IWM", "DIA"]


class DataDownloader:
    """
    DataDownloader is a class designed to fetch historical stock and ETF data.
    
    Attributes:
    - tickers: A list of stock tickers.
    - etfs: A list of ETF tickers.
    
    Methods:
    - download(period): Downloads historical data for the given period. Combines tickers and etfs.
    - quarterly(): Downloads data for the past 3 months.
    - annual(): Downloads data for the past year.
    - decade(): Downloads data for the past 10 years.
    
    """
    def __init__(self, tickers, etfs):
        self.tickers = tickers
        self.etfs = etfs

    def download(self, period='1y'):
        return yf.download(self.tickers + self.etfs, period=period)

    def quarterly(self):
        return self.download(period='3mo')

    def annual(self):
        return self.download(period='1y')
    
    def decade(self):
        return self.download(period='10y')
    

"""
After defining the class, an instance of the DataDownloader is created using the provided tickers and ETFs. 
The historical data for the tickers and ETFs is then fetched for the three time periods and stored in 
`quarterly_data`, `annual_data`, and `decade_data`.
"""
downloader = DataDownloader(tickers, etfs)
# Usage Guide:
quarterly_data = downloader.quarterly()
annual_data = downloader.annual()
decade_data = downloader.decade()


# Trading days = count of rows per period
TRADING_DAYS_QUARTER = quarterly_data['Adj Close']['NVDA'].count()
TRADING_DAYS_ANNUAL = annual_data['Adj Close']['NVDA'].count()

def calculate_weights(prices):
    total_price = prices.sum()
    return (prices / total_price) * 100

def calculate_annualized_volatility(ticker_data):
    daily_return = ticker_data.pct_change().dropna()
    std_dev = daily_return.std()
    return std_dev * math.sqrt(TRADING_DAYS_QUARTER) * 100

def calculate_beta(stock, etf, returns):
    covariance = returns[stock].cov(returns[etf])
    variance = returns[etf].var()
    return str(round(covariance / variance,2))

# 1.Create table
part_one = pd.DataFrame(index=tickers)

# 2.Calculate weights
part_one['Weight'] = calculate_weights(quarterly_data['Adj Close'][tickers].iloc[-1]).round(2).astype(str) + "%"

# 3.Calculate annualized volatility
part_one['Annualized Volatility'] = quarterly_data['Adj Close'][tickers].apply(calculate_annualized_volatility).round(2).astype(str) + "%"

# 4-6.Calculate beta
returns = annual_data['Adj Close'].pct_change().dropna()
for etf in etfs:
    table[f'Beta against {etf}'] = [calculate_beta(stock, etf, returns) for stock in tickers]

# 7-8.Calculate average and maximum weekly drawdown
highs = annual_data['High'][tickers].pct_change().dropna()
lows = annual_data['Low'][tickers].pct_change().dropna()
part_one['Average Weekly Drawdown'] = (((lows.mean() - highs.mean()) / highs.mean()) * 100).round(2)
part_one['Maximum Weekly Drawdown'] = (((lows.min() - highs.max()) / highs.max()) * 100).round(2)

# 9-10.Calculate total return and annualized total return
initial_values = decade_data['Adj Close'][tickers].dropna().iloc[0]
current_values = decade_data['Adj Close'][tickers].iloc[-1]
part_one['Total Return'] = (((current_values - initial_values) / initial_values) * 100).round(2).astype(str) + "%"
part_one['Annualized Total Return'] = ((((current_values / initial_values) ** (1/10)) - 1) * 100).round(2).astype(str) + "%"

#new table called part_two
part_two = pd.DataFrame(index= etfs)

# Part 2: Correlation agaisnt ETF's
annual_data_complete = annual_data['Adj Close'].pct_change().drop(annual_data_change.index[0])

corr_against_etf_list = []
for etf in etfs:
    annual_data_complete['portfolio'] = annual_data_complete[tickers].sum(axis=1) 
    corr_against_etf = annual_data_complete[etf].corr(annual_data_complete['portfolio'])
    corr_against_etf_list.append(round(corr_against_etf,2))

part_two['Correlation against ETF'] = corr_against_etf_list

# Part 3: Covariance agaisnt ETF's
cov_against_etf_list = []
for etf in etfs:
    annual_data_complete['portfolio'] = annual_data_complete[tickers].sum(axis=1) 
    cov_against_etf = annual_data_complete[[etf , 'portfolio']].cov()
    cov_against_etf_list.append(round(cov_against_etf[etf]['portfolio']*100,2))

part_two['Covariance against ETF'] = cov_against_etf_list

# Part 4: Tracking Error (10 years)

tracking_errors = []
for etf in etfs:
    decade_data['total_pct_change_tickers'] = decade_data['Adj Close'][tickers + [etf]].sum(axis=1)
    decade_data = decade_data.drop(decade_data.index[0])
    decade_data['TE'] = decade_data['total_pct_change_tickers'] - decade_data['Adj Close'][etf]
    
    te = (decade_data['TE'].std() * 100) * math.sqrt(TRADING_DAYS_ANNUAL)
    tracking_errors.append(round(te,2))
    
part_two['Tracking Errors'] = tracking_errors
part_two


# Part 5: Sharpe Ratio
_1_year_data = annual_data['Adj Close'].pct_change().drop(annual_data.index[0])
risk_free_rate = 0 # common to assume risk free rate is 0 

sharpe_ratios = []
for etf in etfs:
    _1_year_data['total_pct_change_tickers'] = _1_year_data[tickers + [etf]].sum(axis=1)
    daily_return_mean = _1_year_data['total_pct_change_tickers'].mean()
    daily_return_stdev = _1_year_data['total_pct_change_tickers'].std()
    
    sharpe_ratio = (daily_return_mean / daily_return_stdev) * 100
    sharpe_ratios.append(round(sharpe_ratio,2))

part_two['Sharpe Ratio'] = sharpe_ratios
part_two

#Part: 6 - Annualized Volatility

ann_vol_spreads = []
for etf in etfs:
    # port voli
    _1_year_data['total_pct_change'] = _1_year_data[tickers].sum(axis=1)
    ann_port_daily_voli = (_1_year_data['total_pct_change'].std()) * math.sqrt(252)


    # etf voli 
    _1_year_data['total_pct_change'] = _1_year_data[etf]
    ann_etf_daily_voli = (_1_year_data['total_pct_change'].std()) * math.sqrt(252)

    ann_vol_spreads.append(round((ann_port_daily_voli - ann_etf_daily_voli) * 100,2))
    
part_two['Annualized Volatility Spread'] = ann_vol_spreads
part_two

[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


,Correlation against ETF,Covariance against ETF,Tracking Errors,Sharpe Ratio,Annualized Volatility Spread
SPY,0.79,0.13,630174.48,4.96,228.71
IWM,0.72,0.16,630141.63,4.64,224.09
DIA,0.62,0.09,630107.28,4.98,231.30
